In [1]:
import urllib.request
import os.path
import pathlib
import pandas as pd
import numpy as np

from nameparser import HumanName
import humanize


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [2]:
if not os.path.isfile("2021_Contributions.csv"):
    url = "https://www.nyccfb.info/DataLibrary/2021_Contributions.csv"
    urllib.request.urlretrieve(url, "2021_Contributions.csv")

In [3]:
donations = pd.read_csv("2021_Contributions.csv", header=0, dtype={ 'NAME': str, 'STRNO': str, 'STRNAME': str, 'APARTMENT': str , 'INTZIP': str, 'EXEMPTCD': str})

In [4]:
donations

,ELECTION,OFFICECD,RECIPID,CANCLASS,RECIPNAME,COMMITTEE,FILING,SCHEDULE,PAGENO,SEQUENCENO,REFNO,DATE,REFUNDDATE,NAME,C_CODE,STRNO,STRNAME,APARTMENT,BOROUGHCD,CITY,STATE,ZIP,OCCUPATION,EMPNAME,EMPSTRNO,EMPSTRNAME,EMPCITY,EMPSTATE,AMNT,MATCHAMNT,PREVAMNT,PAY_METHOD,INTERMNO,INTERMNAME,INTSTRNO,INTSTRNM,INTAPTNO,INTCITY,INTST,INTZIP,INTEMPNAME,INTEMPSTNO,INTEMPSTNM,INTEMPCITY,INTEMPST,INTOCCUPA,PURPOSECD,EXEMPTCD,ADJTYPECD,RR_IND,SEG_IND,INT_C_CODE
0,2021,1,2563,P,"Tirschwell, Sara A",H,7,ABC,NaN,NaN,R0000209,3/6/2021,NaN,"""Dombrowski,"", Ray",IND,NaN,NaN,NaN,NaN,Haverford,PA,19041,Turnaround Manager,Alvarez & Marsal,NaN,600 Madison Avenue,New York,NY,2000.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
1,2021,55,2345,P,"Camarena, Rodrigo G",H,6,ABC,NaN,NaN,R0001664,12/17/2020,NaN,"(Dougherty) LoBianco, Kelly",IND,NaN,NaN,NaN,Z,Los Angeles,CA,90068,Chief Program Officer,The HOPE Program,1,Smith Street,Brooklyn,NY,38.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
2,2021,55,2414,P,"Low, Jenny L",H,6,ABC,NaN,NaN,R0000080,9/5/2020,NaN,"(Eng) Lawton, Linnit",IND,NaN,NaN,NaN,K,Brooklyn,NY,11201,Human Resources,Hamilton Madison House,253,South Street,New York,NY,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
3,2021,55,283,P,"Gennaro, James F",M,7,ABC,NaN,NaN,R0002295,3/3/2021,NaN,"(Orlow) Husarsky, Miriam",IND,NaN,NaN,NaN,Q,Richmond Hill,NY,11418,Speech Therapist,NYC Dept of Education,63-55,102nd street,Rego Park,NY,175.0,175,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
4,2021,55,2454,P,"Boghosian Murphy, Leslie",H,6,ABC,NaN,NaN,R0000171,10/19/2020,NaN,"(Zimmerman)Shaich, Carol",IND,NaN,NaN,NaN,M,Ny,NY,10027,Retired,NaN,NaN,NaN,NaN,NaN,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285576,2021,11,2546,P,"Yang, Andrew",H,7,ABC,NaN,NaN,R0029434,1/13/2021,NaN,"Zymaris, Mai",IND,NaN,NaN,NaN,Z,BRAINTREE,MA,2184,Attorney,Morrison & Foerster,428,John Mahar Hwy Unit 207,BRAINTREE,MA,50.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
285577,2021,11,2393,P,"Wiley, Maya D",H,7,ABC,NaN,NaN,R0034934,2/13/2021,NaN,"Zysermann, Noemie",IND,NaN,NaN,NaN,M,New York,NY,10023,Consultant,Self Employed,266,W 73rd st,New York,NY,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
285578,2021,33,1559,P,"Johnson, Corey D",L,3,ABC,NaN,NaN,R0016683,7/11/2019,NaN,"Zyskowska, Celina",IND,NaN,NaN,NaN,Q,Kew Gardens,NY,11415,Accounting,Commodore Construction,602,South 3rd Avenue,Mt Vernon,NY,250.0,250,0.0,2,16.0,"Oneill, Brendan",NaN,NaN,NaN,Long Island City,NY,11104,Commodore Construction,602,South 3rd Avenue,Mt Vernon,NY,Labor Manager,NaN,NaN,NaN,N,N,IND
285579,2021,11,1545,P,"Adams, Eric L",L,1,ABC,NaN,NaN,R0003574,6/12/2018,NaN,"Zyskowski, Joseph",IND,NaN,NaN,NaN,Q,Middle Village,NY,11379,Owner,K Construction,65-58,Admiral Avenue,Middle Village,NY,5000.0,250,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN


In [5]:
major_candidates = [
    'Adams, Eric L',
    'Donovan, Shaun',
    'Garcia, Kathryn A',
    'McGuire, Raymond J',
    'Morales, Dianne',
    'Stringer, Scott M',
    'Wiley, Maya D',
    'Yang, Andrew'
]

mayor_candidates_names = donations[donations['OFFICECD'].isin([1, 11])]['RECIPNAME'].unique()
candidate_last_names = dict(zip(mayor_candidates_names, map(lambda s: HumanName(s).last, mayor_candidates_names)))
candidate_last_names.values()

dict_values(['Tirschwell', 'Wiley', 'Yang', 'Morales', 'Stringer', 'Cullen', 'Garcia', 'McGuire', 'Donovan', 'Menchaca', 'Adams', 'Diaz', 'Pepitone', 'Taylor', 'Sutton', 'Rose', 'Mateo', 'Guimaraes', 'Chang', 'Sliwa', 'Wright', 'Emilien', 'Filipchenko', 'Laurel-Smith', 'Foldenauer', 'Kavovit', 'Francis', 'Prince', 'Nunez', 'Kaplan', 'Oremland', 'Miles', 'Reaves', 'Coenen', 'Bunea', 'Pinto', 'Downs', 'Flores', 'Fitzgerald', 'Krietchman', 'Nampiaparampil', "O'Hagan", 'Seidman', 'Seely'])

In [6]:
# mayor_donations = donations[ (donations['OFFICECD'].isin([1, 11]) & donations[donations["RECIPNAME"].isin(major_candidates) & donations["SCHEDULE"].eq("ABC") ]
mayor_donations = donations[ donations['OFFICECD'].isin([1, 11]) & donations["RECIPNAME"].isin(major_candidates) & donations["SCHEDULE"].eq("ABC") ].copy()

# formate employer

def employed_format(s):
    if not isinstance(s, str):
        return s
    if s.lower() == 'not employed' or s.lower() == 'unemployed':
        return 'Not Employed'
    elif  s.lower() in ['self-employed', 'self employed', 'self', 'Self Employee']:
        return 'Self Employed'
    elif s.lower() == 'none':
        return 'None'
    elif s.lower() in ['homemaker', 'housewife']:
        return 'Homemaker'
    elif s.lower() in ['Citigroup', 'citi']:
        return 'Citigroup'
    elif 'paul weiss' in s.lower():
        return 'Paul Weiss'
    elif ('Skadden Arps' in s.lower() or s == 'Skadden'):
        return 'Skadden Arps'
    elif 'Rosen & Katz' in s or s == 'Wachtell Lipton':
        return 'Wachtell, Lipton, Rosen & Katz'
    else:
        return s


mayor_donations['EMPNAME'] = mayor_donations['EMPNAME'].apply(employed_format)


In [7]:
len(mayor_donations)
1500/len(mayor_donations) * 100

1.8236416908805757

In [8]:
mayor_donations.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82253 entries, 24 to 285579
Data columns (total 52 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ELECTION    82253 non-null  int64  
 1   OFFICECD    82253 non-null  int64  
 2   RECIPID     82253 non-null  object 
 3   CANCLASS    82253 non-null  object 
 4   RECIPNAME   82253 non-null  object 
 5   COMMITTEE   82253 non-null  object 
 6   FILING      82253 non-null  int64  
 7   SCHEDULE    82253 non-null  object 
 8   PAGENO      0 non-null      float64
 9   SEQUENCENO  0 non-null      float64
 10  REFNO       82253 non-null  object 
 11  DATE        82253 non-null  object 
 12  REFUNDDATE  0 non-null      object 
 13  NAME        82253 non-null  object 
 14  C_CODE      82253 non-null  object 
 15  STRNO       0 non-null      object 
 16  STRNAME     0 non-null      object 
 17  APARTMENT   0 non-null      object 
 18  BOROUGHCD   82172 non-null  object 
 19  CITY        82179 non-n

In [9]:
mayor_donations.NAME.value_counts()

Koo, Julia           53
Jordan, Tyron        45
Warren, Roxanne      44
Sherrill, Kenneth    36
Yourke, George       32
                     ..
Kim, Summer           1
Abel, Valere          1
Breakley, Tia         1
Alpert, Ellen         1
Egan, Erin            1
Name: NAME, Length: 55702, dtype: int64

In [10]:
mayor_donations['C_CODE'].unique()

array(['PCOMP', 'CORP', 'IND', 'CAN', 'FAM', 'OTHR', 'PCOMZ', 'EMPO',
       'LLC'], dtype=object)

In [11]:
def parse_name(s):
    names = []
    
    human_name = HumanName(s)
    
    names.append(f"{human_name.last}, {human_name.first}")
    
    if human_name.suffix:
        names.append(f"{human_name.last} {human_name.suffix}, {human_name.first}")
    
    if human_name.middle:
        names.append(f"{human_name.last}, {human_name.first} {human_name.middle}")
    
        if len(human_name.middle) > 1:
            names.append(f"{human_name.last}, {human_name.first} {human_name.middle[0]}")
    
    return names


def name_variations_from_file(file):
    variations = []
    
    with open(file, 'r') as f:
        for line in f:
            for variation in parse_name(line):
                variations.append(variation.upper())

    return variations
                
real_estate_names_from_littlesis = name_variations_from_file('./real_estate_names.txt')
worst_landlords = name_variations_from_file('./worst_landlords.txt')

In [12]:
mayor_donations['realestate_littlesis'] = mayor_donations['NAME'].str.upper().isin(real_estate_names_from_littlesis)
mayor_donations['worst_landlord'] = mayor_donations['NAME'].str.upper().isin(worst_landlords)
mayor_donations['works_in_realestate'] = mayor_donations['OCCUPATION'].str.contains(r'(?i).*REAL ESTATE.*')

In [13]:
flagged_donations = mayor_donations[ (mayor_donations['realestate_littlesis'] | mayor_donations['worst_landlord'] |  mayor_donations['works_in_realestate']  | ( mayor_donations['C_CODE'] == "LLC"))]
flagged_donations_major_candidates = flagged_donations[flagged_donations["RECIPNAME"].isin(major_candidates)]

columns = ['ELECTION', 'RECIPID', 'RECIPNAME', 'COMMITTEE', 'NAME', 'AMNT', 'DATE', 'REFUNDDATE', 'CITY', 'STATE', 'ZIP', 'OCCUPATION', 'EMPNAME', 'C_CODE', 'realestate_littlesis', 'worst_landlord', 'works_in_realestate']

flagged_donations.sort_values(['RECIPNAME', 'DATE'])[columns]

,ELECTION,RECIPID,RECIPNAME,COMMITTEE,NAME,AMNT,DATE,REFUNDDATE,CITY,STATE,ZIP,OCCUPATION,EMPNAME,C_CODE,realestate_littlesis,worst_landlord,works_in_realestate
49376,2021,1545,"Adams, Eric L",L,"Cohen, Adir",5100.0,1/10/2019,NaN,Brooklyn,NY,11230,Real Estate,Renaissance Realty Group,IND,False,False,True
96693,2021,1545,"Adams, Eric L",L,"Gottlieb, Israel",1000.0,1/10/2019,NaN,Lakewood Township,NJ,8701,Real estate investor,Self Employed,IND,False,False,True
164427,2021,1545,"Adams, Eric L",L,"Matheus, Donald",500.0,1/10/2019,NaN,Brooklyn,NY,11238,Real estate,Albatross group,IND,False,False,True
249109,2021,1545,"Adams, Eric L",L,"Strulovitch, Baila",2000.0,1/10/2019,NaN,Brooklyn,NY,11205,Real Estate,Self Employee,IND,False,False,True
148543,2021,1545,"Adams, Eric L",L,"Levy, Danny",100.0,1/10/2020,NaN,Brooklyn,NY,11230,Real estate,Self Employed,IND,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6634,2021,2546,"Yang, Andrew",H,"Alpert, Joseph",1000.0,3/9/2021,NaN,Fort Lee,NJ,7024,Real Estate,Self Employed,IND,False,False,True
19718,2021,2546,"Yang, Andrew",H,"Bellini, Arthur",2000.0,3/9/2021,NaN,New York,NY,10012,Real Estate,Black Bear Asset Management,IND,False,False,True
43526,2021,2546,"Yang, Andrew",H,"Chen, Cindy",10.0,3/9/2021,NaN,Brooklyn,NY,11201,Real estate agent,Compass,IND,False,False,True
85071,2021,2546,"Yang, Andrew",H,"Friedman, Joseph",250.0,3/9/2021,NaN,Brooklyn,NY,11210,Real estate,Self Employed,IND,False,False,True


In [14]:
flagged_donations.sort_values(['RECIPNAME', 'DATE']).to_csv('flagged_donations.csv')

In [15]:
def to_money(i):
 return "$" + humanize.intcomma(round(i))

def to_person_name(s):
    return str(HumanName(s))

In [16]:
total_amount_recieved = mayor_donations[["RECIPNAME", "AMNT"]].groupby("RECIPNAME").sum().sort_values(by="AMNT", ascending=False)
total_amount_recieved["AMNT"]  = total_amount_recieved["AMNT"].apply(to_money)
total_amount_recieved = total_amount_recieved.rename_axis('Candidates').rename(columns={'AMNT': 'Amount'})
total_amount_recieved

,Amount
Candidates,
"McGuire, Raymond J","$7,645,510"
"Adams, Eric L","$3,807,982"
"Stringer, Scott M","$2,555,446"
"Donovan, Shaun","$2,320,507"
"Yang, Andrew","$2,174,180"
"Wiley, Maya D","$1,057,497"
"Garcia, Kathryn A","$586,578"
"Morales, Dianne","$562,751"


In [17]:
def largest_donors_per_candidate(donations, limit=15):
    agg = donations[["RECIPNAME", "AMNT", "NAME", "EMPNAME"]].groupby(["RECIPNAME", "NAME"]).agg({'AMNT': sum, 'EMPNAME': lambda s: s.unique() }) # .sort_values(by='AMNT', ascending=False) # .groupby(level=0, group_keys=False).head(20) # .reset_index() # ort_values(by='AMNT')  # .groupby(level=0, group_keys=False).apply(lambda x: x.sort_values(('AMNT', 'sum'), ascending=False))
        
    
    return agg['AMNT'].groupby('RECIPNAME', group_keys=False).apply(lambda x: x.sort_values(ascending=False).head(limit).apply(to_money)).to_frame()
    

In [18]:
largest_donors_per_candidate(mayor_donations)

AMNT
RECIPNAME          NAME                                
Adams, Eric L      Bensusan, Danny              $12,100
                   Waisman, Shai                $10,200
                   Guzel, Murat                 $10,100
                   Boutross, Diana              $10,000
                   Singh, Navdeep               $10,000
                   Serova, Daria                $10,000
                   Bartlett, Gregory             $7,600
                   Akcetin, Mehmet               $7,000
                   David, Emanuel                $6,600
                   Cerchione, Gregory            $6,000
                   Razi, Sami Uddin              $6,000
                   Basusta, Yusuf                $5,500
                   Daneshvar, Michelle           $5,350
                   engelbrecht, Kate             $5,100
                   Belt, Antonia                 $5,100
Donovan, Shaun     Donovan, Shaun               $19,100
                   Blumenstein, Ruth            $15,300
                   Briganti, Stephen            $10,200
                   Zabar, Eli                   $10,200
                   Nolen, Eliot                 $10,200
                   Rockefeller, David           $10,200
                   Fredericks, Devon            $10,200
                   Bingham, Clara               $10,200
                   Goodman, Sara                $10,150
                   Niejelow, Alexander           $8,100
                   Hickenlooper, Robin           $8,000
                   Whang, John                   $6,100
                   Meyer, Audrey                 $5,600
                   Werner, Zachary               $5,600
                   Donovan, Lucas                $5,450
Garcia, Kathryn A  Smith, Daniel                 $2,250
                   sando, vicki                  $2,135
                   Hughes, Chris                 $2,000
                   Berardi, Eugene               $2,000
                   Wilson, Frederick             $2,000
                   Hansen, Monika                $2,000
                   Glazer, Elizabeth             $2,000
                   Wachsberger, Chaim            $2,000
                   Wilson, Joanne                $2,000
                   Kerr, Zachary                 $2,000
                   Kelly, Philip                 $2,000
                   MacArthur, Andrew             $2,000
                   Russo, Michael                $2,000
                   Uniformed Sanitation Chiefs   $2,000
                   Lettieri, Greg                $2,000
McGuire, Raymond J Williams-Isom, Anne          $15,300
                   Rubin, Robert                $10,200
                   Tisch, James                 $10,200
                   Lutnick, Allison             $10,200
                   Gerson, Mark                 $10,200
                   Rose, Joe                    $10,200
                   Abramson, Carrie             $10,200
                   Fuller, Vicki                $10,200
                   Adjmi, Marcus                $10,200
                   Ross, Kara                   $10,200
                   Chenault, Kenneth            $10,200
                   Atkins, Peter                $10,200
                   Wolfe, Lisa                  $10,200
                   Schnell, Paul                $10,200
                   Kempner, Michael             $10,100
Morales, Dianne    Weinberg, Jason               $6,000
                   momeyer, alan                 $2,100
                   Little, Keith                 $2,100
                   Shavers, Frances              $2,030
                   Murray, Brian                 $2,000
                   Kovner, Katherine             $2,000
                   Murray, Edward                $2,000
                   Schnapper, Simeon             $2,000
                   Wainright, Hannah             $2,000
                   Johnson, William              $2,000
                   

In [19]:
largest_donors_per_candidate(mayor_donations, limit=10  )

AMNT
RECIPNAME          NAME                         
Adams, Eric L      Bensusan, Danny       $12,100
                   Waisman, Shai         $10,200
                   Guzel, Murat          $10,100
                   Boutross, Diana       $10,000
                   Singh, Navdeep        $10,000
                   Serova, Daria         $10,000
                   Bartlett, Gregory      $7,600
                   Akcetin, Mehmet        $7,000
                   David, Emanuel         $6,600
                   Cerchione, Gregory     $6,000
Donovan, Shaun     Donovan, Shaun        $19,100
                   Blumenstein, Ruth     $15,300
                   Briganti, Stephen     $10,200
                   Zabar, Eli            $10,200
                   Nolen, Eliot          $10,200
                   Rockefeller, David    $10,200
                   Fredericks, Devon     $10,200
                   Bingham, Clara        $10,200
                   Goodman, Sara         $10,150
                   Niejelow, Alexander    $8,100
Garcia, Kathryn A  Smith, Daniel          $2,250
                   sando, vicki           $2,135
                   Hughes, Chris          $2,000
                   Berardi, Eugene        $2,000
                   Wilson, Frederick      $2,000
                   Hansen, Monika         $2,000
                   Glazer, Elizabeth      $2,000
                   Wachsberger, Chaim     $2,000
                   Wilson, Joanne         $2,000
                   Kerr, Zachary          $2,000
McGuire, Raymond J Williams-Isom, Anne   $15,300
                   Rubin, Robert         $10,200
                   Tisch, James          $10,200
                   Lutnick, Allison      $10,200
                   Gerson, Mark          $10,200
                   Rose, Joe             $10,200
                   Abramson, Carrie      $10,200
                   Fuller, Vicki         $10,200
                   Adjmi, Marcus         $10,200
                   Ross, Kara            $10,200
Morales, Dianne    Weinberg, Jason        $6,000
                   momeyer, alan          $2,100
                   Little, Keith          $2,100
                   Shavers, Frances       $2,030
                   Murray, Brian          $2,000
                   Kovner, Katherine      $2,000
                   Murray, Edward         $2,000
                   Schnapper, Simeon      $2,000
                   Wainright, Hannah      $2,000
                   Johnson, William       $2,000
Stringer, Scott M  Guenther, Diane A     $10,200
                   Klein, John S         $10,100
                   Redhead, Fabian        $9,900
                   Quinlan, Robert        $7,900
                   Zeckendorf, Arthur     $7,500
                   Cohen, Steven          $5,450
                   Gelman, Audrey         $5,450
                   Sinensky, Michael      $5,350
                   Mehiel, Karen          $5,350
                   Koeppel, Howard        $5,200
Wiley, Maya D      Essex, Andrew          $4,100
                   Carter, Zachary        $4,000
                   Wang, Elizabeth        $3,500
                   Pinderhughes, Dianne   $3,150
                   Bluhdorn, Dominique    $2,750
                   Tamagni, Jordan        $2,500
                   Winslow, Barbara       $2,400
                   Hershenov, Eileen      $2,250
                   Leif, Carol            $2,250
                   Messing, Debra         $2,250
Yang, Andrew       Koo, Julia             $8,877
                   Hu, Jordan             $4,000
                   DiCristo, Aurelia      $4,000
                   Nguyen, Kevin          $4,000
                   Emerson, Holly         $4,000
                   Sun, Yeren             $4,000
                   Diaco, Anthony         $4,000
                   Bergman, Barry         $4,000
                   Cornick, Kenneth       $4,000
                   Lai, Chao Cheng        $3,750

In [20]:
# Largest Donors
largest_donors_per_candidate(mayor_donations)

AMNT
RECIPNAME          NAME                                
Adams, Eric L      Bensusan, Danny              $12,100
                   Waisman, Shai                $10,200
                   Guzel, Murat                 $10,100
                   Boutross, Diana              $10,000
                   Singh, Navdeep               $10,000
                   Serova, Daria                $10,000
                   Bartlett, Gregory             $7,600
                   Akcetin, Mehmet               $7,000
                   David, Emanuel                $6,600
                   Cerchione, Gregory            $6,000
                   Razi, Sami Uddin              $6,000
                   Basusta, Yusuf                $5,500
                   Daneshvar, Michelle           $5,350
                   engelbrecht, Kate             $5,100
                   Belt, Antonia                 $5,100
Donovan, Shaun     Donovan, Shaun               $19,100
                   Blumenstein, Ruth            $15,300
                   Briganti, Stephen            $10,200
                   Zabar, Eli                   $10,200
                   Nolen, Eliot                 $10,200
                   Rockefeller, David           $10,200
                   Fredericks, Devon            $10,200
                   Bingham, Clara               $10,200
                   Goodman, Sara                $10,150
                   Niejelow, Alexander           $8,100
                   Hickenlooper, Robin           $8,000
                   Whang, John                   $6,100
                   Meyer, Audrey                 $5,600
                   Werner, Zachary               $5,600
                   Donovan, Lucas                $5,450
Garcia, Kathryn A  Smith, Daniel                 $2,250
                   sando, vicki                  $2,135
                   Hughes, Chris                 $2,000
                   Berardi, Eugene               $2,000
                   Wilson, Frederick             $2,000
                   Hansen, Monika                $2,000
                   Glazer, Elizabeth             $2,000
                   Wachsberger, Chaim            $2,000
                   Wilson, Joanne                $2,000
                   Kerr, Zachary                 $2,000
                   Kelly, Philip                 $2,000
                   MacArthur, Andrew             $2,000
                   Russo, Michael                $2,000
                   Uniformed Sanitation Chiefs   $2,000
                   Lettieri, Greg                $2,000
McGuire, Raymond J Williams-Isom, Anne          $15,300
                   Rubin, Robert                $10,200
                   Tisch, James                 $10,200
                   Lutnick, Allison             $10,200
                   Gerson, Mark                 $10,200
                   Rose, Joe                    $10,200
                   Abramson, Carrie             $10,200
                   Fuller, Vicki                $10,200
                   Adjmi, Marcus                $10,200
                   Ross, Kara                   $10,200
                   Chenault, Kenneth            $10,200
                   Atkins, Peter                $10,200
                   Wolfe, Lisa                  $10,200
                   Schnell, Paul                $10,200
                   Kempner, Michael             $10,100
Morales, Dianne    Weinberg, Jason               $6,000
                   momeyer, alan                 $2,100
                   Little, Keith                 $2,100
                   Shavers, Frances              $2,030
                   Murray, Brian                 $2,000
                   Kovner, Katherine             $2,000
                   Murray, Edward                $2,000
                   Schnapper, Simeon             $2,000
                   Wainright, Hannah             $2,000
                   Johnson, William              $2,000
                   

In [21]:
# Largest Real Estate Donors
largest_donors_per_candidate(flagged_donations_major_candidates, limit=30)

AMNT
RECIPNAME          NAME                              
Adams, Eric L      Boutross, Diana            $10,000
                   tamir, Elliot               $5,100
                   Berman, Nathan              $5,100
                   Verrone, Robert             $5,100
                   Tawil, Eliot                $5,100
                   Demirjian, Alexis           $5,100
                   Cohen, Adir                 $5,100
                   Borrok, Charles R.          $5,100
                   Shurka, Manny               $5,000
                   Schwartz, Yoel              $5,000
                   Corman, Jeffrey             $5,000
                   Landau, Solomon             $5,000
                   Neamonitakis, Michael       $4,000
                   Ward, Jesse                 $3,400
                   Vitucci Sr, Dominick        $3,400
                   Fisher, Avi                 $3,400
                   Silviano, John              $3,000
                   Anagnost, Dick              $3,000
                   Levkov, Viacheslav          $3,000
                   Gindi, jeffrey              $2,675
                   Iannucci, Robert            $2,500
                   Ewers, Sonia                $2,500
                   Marchi, Dean                $2,500
                   Chassin, Jonathan           $2,500
                   Sciortino, Anthony          $2,500
                   Basile, Rocco               $2,250
                   Ghershenzon, Mordechai      $2,000
                   Chalme, Raymond             $2,000
                   Sanjana, Aroza              $2,000
                   Cohen, Meir                 $2,000
Donovan, Shaun     Fisher, Winston             $5,100
                   DeBode, Gary                $5,100
                   olshan, andrea              $5,100
                   Bahar, Aaron                $5,100
                   KATZ, JEFFREY               $5,100
                   Moore, John                 $5,100
                   Sheller, Chris              $5,100
                   Bernstein, Joshua           $5,100
                   Silvers, Nicholas           $5,100
                   Brodsky, Daniel             $5,100
                   Silverstein, Larry          $5,100
                   Stacom, Darcy               $5,100
                   Zeiler, John                $5,100
                   Winn, Arthur                $5,100
                   Clark, Ric                  $5,100
                   Martin, Rob                 $3,000
                   Rubler, Neil                $2,500
                   Benenson, Lawrence          $2,500
                   kuriloff, josh              $2,500
                   Wiener, Alan                $2,250
                   Savitt, Robert              $2,000
                   Wiener, Matthew             $2,000
                   Kerr, Zachary               $2,000
                   Willard, Sara               $1,750
                   Murray, Brian               $1,500
                   Mandel, Barry               $1,500
                   Happel, Michael             $1,100
                   Fisher, Beth                $1,000
                   Shapiro, Ben                $1,000
                   Farman-Farmaian, Patricia   $1,000
Garcia, Kathryn A  Kalikow, Peter              $2,000
                   Kerr, Zachary               $2,000
                   Durst, Helena               $2,000
                   MacArthur, Andrew           $2,000
                   Ezra, Brian                 $1,400
                   Wark, Jesse                 $1,050
                   Peterson, Douglas             $750
                   Simkowitz, Chaim              $750
                   Ghossein, Yara                $750
                   Fruchthandler, Efraim         $750
                   Nigrelli, Randy               $700
                   Amiri, Behzad                 $500
                   Cottingham, Martin            $500

In [22]:
def real_estate_donors_for(candidate):
        columns = ['RECIPNAME', 'DATE', 'AMNT', 'NAME', 'ZIP', 'OCCUPATION', 'EMPNAME']
        donations = flagged_donations_major_candidates[flagged_donations_major_candidates['RECIPNAME'] == candidate][columns]
        donations["NAME"] =  donations["NAME"].apply(to_person_name)
        return donations.sort_values('AMNT', ascending=False)

real_estate_donors_for('Adams, Eric L').head(50)

,RECIPNAME,DATE,AMNT,NAME,ZIP,OCCUPATION,EMPNAME
27270,"Adams, Eric L",4/26/2018,5100.0,Charles R. Borrok,10013,Real Estate Broker,Cushman & Wakefield
252074,"Adams, Eric L",7/8/2018,5100.0,Elliot tamir,11230,Real Estate,Vesper Holdings
264605,"Adams, Eric L",8/13/2018,5100.0,Robert Verrone,7078,Real Estate Finance,Self Employed
21859,"Adams, Eric L",7/10/2018,5100.0,Nathan Berman,10128,Real Estate Development,Metro Loft Management
49376,"Adams, Eric L",1/10/2019,5100.0,Adir Cohen,11230,Real Estate,Renaissance Realty Group
253092,"Adams, Eric L",6/26/2019,5100.0,Eliot Tawil,11230,Real Estate,Wharton Preperties
62376,"Adams, Eric L",6/29/2018,5100.0,Alexis Demirjian,10023,Real Estate Broker,Douglas Elliman
236648,"Adams, Eric L",5/7/2018,5000.0,Manny Shurka,11050,Commercial Real Estate,Signature Investment Group
53061,"Adams, Eric L",1/11/2019,5000.0,Jeffrey Corman,11215,Real Estate Investor,Self Employee
230492,"Adams, Eric L",6/11/2018,5000.0,Yoel Schwartz,11206,Real estate investor,Blackdiam Properties LLC


In [23]:
len(flagged_donations[flagged_donations['RECIPNAME'] == 'Adams, Eric L']['NAME'].unique())

247

In [24]:
pathlib.Path("donations").mkdir(parents=True, exist_ok=True)

for cand in ['Adams, Eric L', 'Donovan, Shaun', 'Garcia, Kathryn A', 'McGuire, Raymond J', 'Stringer, Scott M', 'Yang, Andrew']:
    csv_name = f"donations/{candidate_last_names.get(cand)}_top_realestate_donors.csv"
    top_donations = real_estate_donors_for(cand).head(25).to_csv(csv_name)


In [25]:
largest_donors_per_candidate(flagged_donations_major_candidates, limit=3)

AMNT
RECIPNAME          NAME                       
Adams, Eric L      Boutross, Diana     $10,000
                   tamir, Elliot        $5,100
                   Berman, Nathan       $5,100
Donovan, Shaun     Fisher, Winston      $5,100
                   DeBode, Gary         $5,100
                   olshan, andrea       $5,100
Garcia, Kathryn A  Kalikow, Peter       $2,000
                   Kerr, Zachary        $2,000
                   Durst, Helena        $2,000
McGuire, Raymond J Colacino, Michael    $8,433
                   winter, benjamin     $5,100
                   Goldberg, Andrew     $5,100
Morales, Dianne    Rosenthal, Richard     $500
                   Neveloff, Jay          $400
                   Fried, Daniel          $350
Stringer, Scott M  Klein, John S       $10,100
                   Quinlan, Robert      $7,900
                   Brodsky, Daniel      $5,100
Wiley, Maya D      Khan, Fazeel         $2,000
                   Johnson, Michael     $1,000
                   Zelermyer, Karen       $750
Yang, Andrew       Korein, James        $2,000
                   Wei, Nelson          $2,000
                   DeLilli, Mark        $2,000

In [26]:
def top_donors_for(df, head=20):
    top_donors = df.groupby(["NAME"]).agg({"AMNT": 'sum', 'RECIPNAME': lambda series: list(series.unique()) }).sort_values(by="AMNT", ascending=False).head(head).reset_index()
    top_donors.rename(columns={"NAME": "Donor Name", "AMNT": "Amount", "RECIPNAME": "Supported Candidates" }, inplace=True)
    top_donors['Donor Name'] = top_donors['Donor Name'].apply(lambda x: str(HumanName(x)))
    top_donors["Amount"] = top_donors["Amount"].apply(lambda x: f"${humanize.intcomma(round(x))}" )
    top_donors["Supported Candidates"] = top_donors["Supported Candidates"].apply(lambda names: ", ".join(map(candidate_last_names.get, names)))
    return top_donors


all_top_donors = top_donors_for(mayor_donations, head=50)
top_real_estate_donors = top_donors_for(flagged_donations, head=50)


In [27]:
mayor_donations[["EMPNAME", "AMNT"]].groupby(["EMPNAME"]).agg(['sum', 'count']).sort_values(by=('AMNT', 'sum'), ascending=False).head(100)


AMNT       
                                       sum  count
EMPNAME                                          
Not Employed                    3590179.52  24050
Self Employed                   2289462.03   6765
Retired                          507235.55   1322
Paul Weiss                       205321.42    113
Citigroup                        147057.06    187
Homemaker                        108975.00    185
Skadden Arps                     100905.00     98
Self Employee                     72810.00    108
Wachtell, Lipton, Rosen & Katz    70245.00     45
None                              66006.81     73
Pershing Square Capital Manage    39200.00      9
Monroe College                    38075.00     34
Google                            37437.60    252
Columbia University               37135.84    265
Sullivan & Cromwell LLP           36860.00     30
Lazard                            36600.00      9
Kirkland & Ellis LLP              35145.00     25
City of New York                  34269.06    432
Skadden Arps Slate Meagher & F    33800.00     17
NYU                               31393.69    232
Cravath, Swaine & Moore LLP       31175.00     17
retired                           30470.00     87
Centerview Partners               29500.00      7
Kirkland & Ellis                  29400.00     16
NYPD                              26644.00    228
Compass                           24696.21     60
Mediaocean                        24161.00     46
Long Pond Capital                 23900.00      6
PJT Partners                      23600.00      9
Standard Industries               21800.00      7
Cleary Gottlieb Steen & Hamilt    21550.00     14
Willkie Farr & Gallagher LLP      21218.00     22
Cleary Gottlieb                   21100.00     14
JLL                               20535.00     23
Fisher Brothers                   20435.00      6
CBRE                              19825.00     16
Bank of America                   19557.42     61
Fordham University                19198.42     49
Morgan Stanley                    18980.21     40
ViacomCBS                         18947.21     45
Student                           17966.07     76
NYC DOE                           17607.22    340
Shearman & Sterling LLP           17292.70     14
King & Spalding LLP               17275.00     12
King & Spalding                   16900.00     11
Mastercard                        16595.00     27
Tashkent Supermarket              16000.00      7
Douglas Elliman                   15825.00     26
DSNY                              15773.07    174
Estee Lauder Companies            15500.00      6
Fairstead                         15300.00      3
TG Holdings, LLC                  15300.00      3
Allen & Company LLC               15300.00      3
Rubenstein Communications         15150.00      3
Simpson Thacher & Bartlett LLP    15000.00     15
Ehrenkranz Partners               14875.00      6
Harvard Business School           13950.00      8
Spencer Stuart                    13265.00      8
Wachtell, Lipton, Rosen & Kat     13100.00      7
Bloomberg                         13000.21     32
cushman and wakefield             12750.00      4
Centerview Partners LLC           12700.00      3
MSG                               12700.00      5
Corcoran                          12355.21     23
The Brodsky Organization          12350.00      3
Netflix                           12206.21     19
Sessa Capital                     12200.00      3
AlleyCorp                         12200.00      3
Loews Corp                        12200.00      3
Willett Advisors LLC              12200.00      3
Bensusan Restaurant Corp          12100.00      3
Gravity Construction              12000.00      3
Sacks & Sacks LLP                 12000.00      6
Lowenstein Sandler LLP            11950.00      6
Katten Muchin Rosenman LLP        11850.00      5
CUNY                              11741.82    187
Acrisure LLC                      11700.00      5
DLA Piper                         1170

In [28]:
top_real_estate_donors

,Donor Name,Amount,Supported Candidates
0,Daniel Brodsky,"$10,200","Stringer, Donovan"
1,Winston Fisher,"$10,200","Stringer, Donovan"
2,John S Klein,"$10,100",Stringer
3,Diana Boutross,"$10,000",Adams
4,Michael Colacino,"$8,433",McGuire
5,Robert Quinlan,"$7,900",Stringer
6,Peter Kalikow,"$7,100","Garcia, McGuire"
7,Derek Trulson,"$7,100","Adams, McGuire"
8,Robert Verrone,"$7,100","Adams, Yang"
9,Richard Ruben,"$7,100","Adams, McGuire"


In [29]:
all_top_donors

,Donor Name,Amount,Supported Candidates
0,Shaun Donovan,"$19,100",Donovan
1,Anne Williams-Isom,"$17,300","Wiley, McGuire"
2,Ruth Blumenstein,"$15,300",Donovan
3,Steven Rattner,"$14,200","Stringer, Adams, Donovan, McGuire"
4,James Dolan,"$12,200","Adams, Donovan, McGuire"
5,Kevin Ryan,"$12,200","Donovan, McGuire, Yang"
6,James Tisch,"$12,200","Garcia, McGuire"
7,Robert Verrone,"$12,200","Adams, McGuire, Yang"
8,John Petry,"$12,200","Adams, McGuire, Yang"
9,Sara Goodman,"$12,150","Stringer, Donovan"


In [30]:
sum_from_real_estate = flagged_donations_major_candidates[['RECIPNAME', 'AMNT']].groupby('RECIPNAME').sum()
sum_from_real_estate = sum_from_real_estate.sort_values(by='AMNT', ascending=False)
sum_from_real_estate['AMNT'] = sum_from_real_estate['AMNT'].apply(to_money)

sum_from_real_estate

,AMNT
RECIPNAME,
"McGuire, Raymond J","$264,904"
"Adams, Eric L","$234,044"
"Stringer, Scott M","$149,423"
"Donovan, Shaun","$133,311"
"Yang, Andrew","$63,117"
"Garcia, Kathryn A","$30,875"
"Wiley, Maya D","$8,937"
"Morales, Dianne","$2,787"


In [31]:
total_amount_recieved = mayor_donations[["RECIPNAME", "AMNT"]].groupby("RECIPNAME").sum().sort_values(by='RECIPNAME')
total_amount_recieved["Real Estate"] = flagged_donations_major_candidates[['RECIPNAME', 'AMNT']].groupby('RECIPNAME').sum().sort_values(by='RECIPNAME')
total_amount_recieved = total_amount_recieved.reset_index().sort_values(by='Real Estate', ascending=False)

total_amount_recieved["Total"]  = total_amount_recieved["AMNT"].apply(to_money)
total_amount_recieved["Real Estate"] = total_amount_recieved['Real Estate'].apply(to_money)
total_amount_recieved['Name'] = total_amount_recieved['RECIPNAME'].apply(to_person_name)

# print(total_amount_recieved[['Name', 'Total', 'Real Estate']].to_string())

total_amount_recieved[['Name', 'Total', 'Real Estate']].style.hide_index()

Name,Total,Real Estate
Raymond J McGuire,"$7,645,510","$264,904"
Eric L Adams,"$3,807,982","$234,044"
Scott M Stringer,"$2,555,446","$149,423"
Shaun Donovan,"$2,320,507","$133,311"
Andrew Yang,"$2,174,180","$63,117"
Kathryn A Garcia,"$586,578","$30,875"
Maya D Wiley,"$1,057,497","$8,937"
Dianne Morales,"$562,751","$2,787"


In [32]:
to_money(flagged_donations_major_candidates['AMNT'].sum())

'$887,398'

In [33]:
flagged_donations_major_candidates['AMNT'].sum() / mayor_donations['AMNT'].sum() * 100

4.284781966891966

In [34]:
employer_agg = mayor_donations[["RECIPNAME", "EMPNAME", "AMNT"]].copy()

# employer_agg['EMPNAME'] = employer_agg['EMPNAME'].apply(employed_format)

employer_agg = employer_agg[["RECIPNAME", "EMPNAME", "AMNT"]].groupby(["RECIPNAME", "EMPNAME"]).agg(['sum', 'count']).sort_values(by=('AMNT', 'sum'), ascending=False)

In [53]:
# top_employers
top_employers = mayor_donations[["EMPNAME", "AMNT"]].copy()
top_employers['EMPNAME'] = top_employers['EMPNAME'].apply(employed_format)

top_employers = top_employers.groupby('EMPNAME').sum().sort_values('AMNT', ascending=False).reset_index()[['EMPNAME', 'AMNT']].sort_values(by='AMNT', ascending=False)
top_employers['Employer Name'] = top_employers['EMPNAME']
top_employers['Amount'] = top_employers['AMNT'].apply(to_money)

top_employers[['Employer Name', 'Amount']].head(100).to_csv('top_100_employers.csv', index=False)


top_employers[['Employer Name', 'Amount']].head(20)




,Employer Name,Amount
0,Not Employed,"$3,590,180"
1,Self Employed,"$2,289,462"
2,Retired,"$507,236"
3,Paul Weiss,"$205,321"
4,Citigroup,"$147,057"
5,Homemaker,"$108,975"
6,Skadden Arps,"$100,905"
7,Self Employee,"$72,810"
8,"Wachtell, Lipton, Rosen & Katz","$70,245"
9,None,"$66,007"


In [36]:
employer_agg.head(100)

AMNT      
                                                          sum count
RECIPNAME          EMPNAME                                         
McGuire, Raymond J Not Employed                    1359250.41  1695
                   Self Employed                    735490.68   792
Donovan, Shaun     Not Employed                     697154.56  1407
Adams, Eric L      Self Employed                    689157.00  1123
Yang, Andrew       Not Employed                     601950.53  7443
Stringer, Scott M  Not Employed                     398529.56  2640
Wiley, Maya D      Not Employed                     266192.90  6109
McGuire, Raymond J Retired                          260943.85   214
Donovan, Shaun     Self Employed                    260654.84   457
Stringer, Scott M  Self Employed                    223902.96   900
McGuire, Raymond J Paul Weiss                       198140.42    95
Morales, Dianne    Not Employed                     145664.98  3654
McGuire, Raymond J Citigroup                        141982.06   146
Wiley, Maya D      Self Employed                    138777.79  1491
Yang, Andrew       Self Employed                    128945.30   998
Adams, Eric L      Retired                          100324.00   363
McGuire, Raymond J Skadden Arps                      96785.00    83
Garcia, Kathryn A  Not Employed                      91238.58   859
Stringer, Scott M  Retired                           89096.50   530
Adams, Eric L      Self Employee                     72810.00   108
Stringer, Scott M  Homemaker                         64935.00   105
Garcia, Kathryn A  Self Employed                     63615.00   368
McGuire, Raymond J None                              63567.81    40
                   Wachtell, Lipton, Rosen & Katz    58450.00    27
Morales, Dianne    Self Employed                     48918.46   636
McGuire, Raymond J Pershing Square Capital Manage    39200.00     9
Adams, Eric L      Homemaker                         38840.00    75
McGuire, Raymond J Lazard                            36600.00     9
Wiley, Maya D      Retired                           36450.99   178
McGuire, Raymond J Sullivan & Cromwell LLP           35050.00    21
                   Skadden Arps Slate Meagher & F    33800.00    17
Stringer, Scott M  Monroe College                    32400.00    23
McGuire, Raymond J Cravath, Swaine & Moore LLP       31100.00    15
Adams, Eric L      Not Employed                      30198.00   243
McGuire, Raymond J Kirkland & Ellis LLP              28925.00    20
                   Kirkland & Ellis                  26600.00    11
                   Centerview Partners               25500.00     5
Adams, Eric L      NYPD                              25416.00   195
Donovan, Shaun     Mediaocean                        24161.00    46
McGuire, Raymond J Long Pond Capital                 23900.00     6
                   PJT Partners                      23600.00     9
                   Cleary Gottlieb Steen & Hamilt    21450.00    13
                   Standard Industries               19800.00     6
                   Willkie Farr & Gallagher LLP      19350.00    13
                   Cleary Gottlieb                   19100.00     9
                   Compass                           18045.21    19
                   King & Spalding LLP               17100.00    11
                   Shearman & Sterling LLP           16832.70     7
                   King & Spalding                   16600.00     9
Donovan, Shaun     Retired                           16530.00     7
Adams, Eric L      Tashkent Supermarket              16000.00     7
McGuire, Raymond J ViacomCBS                         15545.21    13
Garcia, Kathryn A  DSNY                              15433.07   166
McGuire, Raymond J Fordham University                15400.00     4
                   Fairstead                         15300.00     3
                   TG Holdings, LLC                  15300.00     3
Stringer, Scott M  Fisher Brothers           

In [37]:
def employer_donations(name):    
    return mayor_donations[mayor_donations["EMPNAME"].str.lower() == name.lower()]

def employer_donations_contains(name):    
    return mayor_donations[mayor_donations["EMPNAME"].str.lower().contains(name.lower)]

def employer_donations_summary(name):
    
    recipients = employer_donations(name)[["RECIPNAME", "AMNT"]].groupby('RECIPNAME').sum().reset_index().sort_values('AMNT', ascending=False).to_dict('records')
    recipients = list(map(lambda x: f"{to_person_name(x['RECIPNAME'])} {to_money(x['AMNT'])}", recipients))
    
    return {
        "name": name,
        "sum": employer_donations(name)['AMNT'].sum(),
        "donors": list(map(to_person_name, employer_donations(name)["NAME"].unique())),
        "recipients": recipients
    }



In [38]:
employer_donations_summary("NETFLIX")

{'name': 'NETFLIX',
 'sum': 12206.21,
 'donors': ['Rhonda Adams Medina',
  'Kathie Berlin',
  'Cole Galvin',
  'Grant Goodman',
  'Eric Harvey',
  'Melissa Heller-Seid',
  'Chen Jacobson',
  'Taehoon Kim',
  'Rhonda Medina',
  'Tendo Nagenda',
  'Andrew Nguonly',
  'Lyn Pinezich',
  'Renate Radford',
  'Theodore Sarandos',
  'Zhang Zhuge'],
 'recipients': ['Raymond J McGuire $10,350',
  'Shaun Donovan $1,175',
  'Andrew Yang $331',
  'Scott M Stringer $300',
  'Maya D Wiley $50']}

In [39]:
employer_donations_summary("Kirkland & Ellis")

{'name': 'Kirkland & Ellis',
 'sum': 29400.0,
 'donors': ['reginald brown',
  'Lauren Friedman',
  'Daniel Fuglestad',
  'Jonathan Henes',
  'Melissa Hutson',
  'Jason Kanner',
  'Michael Kim',
  'Peter Leung',
  'Ramiro Ocasio',
  'Jen Perkins',
  'Edward Sassower',
  'Eric Wedel',
  'Erica Williams',
  'Ali Zaidi'],
 'recipients': ['Raymond J McGuire $26,600',
  'Scott M Stringer $2,400',
  'Shaun Donovan $400']}

In [40]:
employer_donations_summary("Paul Weiss")

{'name': 'Paul Weiss',
 'sum': 205321.41999999998,
 'donors': ['Jacob Adlerstein',
  'Allan Arffa',
  'Matthew Barnett',
  'Scott Barshay',
  'Paul Basta',
  'Lynn Bayard',
  'Daniel Beller',
  'Joseph Bial',
  'Bruce Birenboim',
  'H. Christopher Boehning',
  'Peter Borock',
  'David Brown',
  'Jessica Carey',
  'Geoffrey Chepiga',
  'Jay Cohen',
  'Emily Compton',
  'Kelley Cornish',
  'Ariel Deckelbaum',
  'Andrew Ehrlich',
  'Sharon Epperson',
  'Ross Fieldston',
  'Brad Finkelstein',
  'Roberto Finzi',
  'Robert Fleder',
  'Martin Flumenbaum',
  'Harris Freidus',
  'Manuel Frey',
  'Andrew Gaines',
  'Sarah Garcia-Tunon',
  'Michael Gertzman',
  'Eric Goodison',
  'Brian Grieve',
  'Udi Grofman',
  'Bruce Gutenplan',
  'Alan Halperin',
  'Claudia Hammerman',
  'Brian Hermann',
  'Rob Holo',
  'Amran Hussein',
  'Stacy Hwang',
  'Loretta Ippolito',
  'Keith James',
  'Jaren Janghorbani',
  'Meredith Kane',
  'Brad Karp',
  'Brian Kim',
  'Joey Kolatch',
  'Daniel Kramer',
  'David 

In [41]:
employer_donations_summary("Google")

{'name': 'Google',
 'sum': 37562.6,
 'donors': ['Jonathan Adams',
  'Rudi Anggono',
  'Michael Appel',
  'Felipe Archondo',
  'Sarah Armstrong',
  'Chan Baik',
  'Rachel Ball',
  'Kenneth Bandes',
  'Eric Barbera',
  'Alexander Bergersen',
  'Briana Bernstein',
  'Tejpaul Bhatia',
  'Marcus Bischof',
  'James Bradbury',
  'Genevieve Brennan',
  'Jared Brick',
  'Benjamin Brittain',
  'Michelle Brous',
  'Shane Burklow',
  'Brad Burns',
  'John Carpenter',
  'John Cater',
  'Angela Chen',
  'Estee Cheng',
  'Weijian Cheng',
  'Po Chui',
  'Jay Chung',
  'John Chung',
  'Peta Clarke',
  'Galen Corey',
  'Marci Czel',
  "Antonio D'Souza",
  'John Demartino',
  'Karen DeSalvo',
  'Andrea DiBenedetto',
  'David Eisenstat',
  'Hunter Ellenbarger',
  'Matthew Elyanow',
  'Gabriel Ewing',
  'Stephen Fang',
  'Desmond Foo',
  'Alec Fox',
  'Matthew Frazier',
  'Beverly Freeman',
  'TRISTAN G SMITH',
  'Leigh Gallagher',
  'Paul Gamble',
  'Raha Ghassemi',
  'Rob Giampietro',
  'Gabriel Giosia',

In [42]:
employer_donations_summary("Apple")

{'name': 'Apple',
 'sum': 10947.84,
 'donors': ['David Alexander',
  'Nicole Aptekar',
  'Alyse Archer-Coite',
  'Jacey Beaucage',
  'Jason Blevins',
  'Rebecca Bradshaw',
  'Lucy Chen',
  'Liana Ching',
  'Eric Chiu',
  'Francoise Cournoyer',
  'Ibrahim Darden',
  'Alberto De Leon',
  'Jamie DeGrazio',
  'Jason Deming',
  'Joseph Enriquez',
  'Gabriel Galang',
  'Bitsie Hefel',
  'Jamy Hickey',
  'Johnny Huang',
  'Lisa Jackson',
  'Kathryn Jefferson',
  'Jarius Jemmott',
  'Alexis Kaback',
  'Yvette Kang',
  'Lexi Kavanaugh',
  'David Kellough',
  'Ben Koning',
  'Jane Krakowski',
  'Michael LaPane',
  'Jason Leung',
  'kayma Liburd',
  'Christopher Lindor',
  'Marykate Macaluso',
  'Danya Martin',
  'Michael McCord',
  'Piyush Mittal',
  'Manuel Pena',
  'Shaun Phillips',
  'TJ Regno',
  'Valeria Restrepo',
  'Stephanie Rivera-Moss',
  'Oscar Rohena',
  'Karmen Starks',
  'Jeff Vacanti',
  'Mark Xue',
  'Xinying Yu',
  'Xuemei Zhang',
  'Xiaosong Zhou'],
 'recipients': ['Shaun Donov